In [14]:
import copy, sys, pandas as pd
import random 
import time 

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS 

from inferring.InferringDFA import InferringDFA

In [15]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [16]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                 cnt_normal,
                 cnt_with_advice,
                 target=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.cnt_normal = cnt_normal
        self.cnt_with_advice = cnt_with_advice
        self.target = target
        self.mq_reduction = int(
            ((cnt_normal[0] - cnt_with_advice[0][0]) / cnt_normal[0]) * 100)
        self.eq_reduction = int(
            ((cnt_normal[1] - cnt_with_advice[0][1]) / cnt_normal[1]) * 100)

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv}, (mq, eq) = {self.cnt_normal} vs (mq, eq) = {self.cnt_with_advice}, mq_reduction = {self.mq_reduction}%, eq_reduction = {self.eq_reduction}%"
        )
    def get_complete_description(self):
        return self.target.print_complete_description()
    
    def print_target(self):
        print(self.target.print_complete_description())

In [29]:
number_of_itreations = 1

#  Fixed random seeds for reproducibility 
seeds =  [i for i in range(number_of_itreations)]

# The alphabet of the DFA
input_signs = ['a', 'b']

# The bound on the number of states 
max_number_of_states = 20

# The list to aggregate the results
results = []

i = 0 
while i < number_of_itreations: 
    random.seed(seeds[i])
    i += 1
    print(f"iter nr: {i}")

    # Create two random DFAs
    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    dfa2.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
    
    # Run learning of the language of DFA to minimize dfa1 and dfa2 
    d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))   
    d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))
    
    while d1.Q < 2 or d2.Q < 2: 
        # Create two random DFAs
        dfa1 = DFA()
        dfa2 = DFA()
        dfa1.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        dfa2.create_random_dfa(Q=random.randint(max_number_of_states//2,max_number_of_states), input_signs=input_signs)
        
        # Run learning of the language of DFA to minimize dfa1 and dfa2 
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))   
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # Create a convolution DFA based on d1 and d2
    conv_dfa = DFA()
    conv_dfa.create_convolution(dfa1=d1, dfa2=d2) 

    # Learn conv_dfa with advice
    start_t = time.time()
    d, cnt_with_advice, cnt_ex_with_advice = run_learning_process(target=conv_dfa, advice_system=SRS(), check_consistency=True) 
    end_t = time.time()

    print("ex time with AS: ", end_t-start_t)

    d.type = DFA.CONV_DFA

    print(f"|d1.Q| = {d1.Q}, |d2.q| = {d2.Q}, |d.Q| = {d.Q}")

    # Run learning process without advice 
    start_t = time.time()
    _, cnt_normal, _ = run_learning_process(target=d, advice_system=None, check_consistency=False)
    end_t = time.time()

    print("ex time without AS: ", end_t-start_t)

    results.append(Record(d1=d1.Q, d2=d2.Q, conv=d.Q, cnt_normal=cnt_normal, cnt_with_advice=(cnt_with_advice, cnt_ex_with_advice), target=copy.deepcopy(d)))

iter nr: 1
ex time with AS:  1.7513868808746338
|d1.Q| = 13, |d2.q| = 15, |d.Q| = 195
ex time without AS:  3.4120218753814697


In [30]:
results.sort(key=lambda x: x.conv)

# Remove degenerated cases (Conv DFA of size 1)
results = [r for r in results if r.conv != 1]

print(f"ilość wpisów w result: {len(results)}")
for r in results:
    r.print_record()
    # r.print_target()

ilość wpisów w result: 1
|d1| = 13, |d2| = 15, |conv| = 195, (mq, eq) = [96299, 90] vs (mq, eq) = ([20657, 2], 68), mq_reduction = 78%, eq_reduction = 97%


In [31]:
f = open("../../learnlib/examples/src/main/java/de/learnlib/example/DfaMedium.txt", "w")
f.write(results[0].get_complete_description())
f.close()